# 120 Years of Olympic History: Data Wrangling and Visualisation in Pandas

**Module:** MN5813 – Business Analytics  
**Assessment:** Group Report (20%) – Olympics Data Wrangling & Visualisation  
**Candidate Number:** 2610587  

---


## 1. Introduction

The Olympic Games provide a useful long‑run record of international sporting competition. The official athlete–event dataset spans all
Summer and Winter Games from Athens 1896 to Rio 2016 and contains information on athlete demographics, events, medals and national
representation. Because it is relatively rich but still manageable in size, it is well suited to practising practical data wrangling and
visualisation in Python.

The aim of this notebook is to work through the full workflow specified in the MN5813 group assignment:

- load the Olympic athlete–event dataset into a Pandas DataFrame;
- carry out basic data cleaning, including dealing with duplicate rows, missing values and simple type conversions;
- construct a small set of derived variables that make later analysis easier;
- perform the core summary analyses required in the brief; and
- produce a set of clear visualisations that help communicate the main patterns in the data.

The structure of the notebook follows the order of the assessment instructions: data loading, cleaning, wrangling, analysis, visualisation,
export of results and concluding remarks.


## 2. Context and brief literature background

Olympic results are often used in teaching examples and in applied research because they sit at the intersection of sport, economics and
politics. A common theme in the literature is that medal counts are heavily concentrated in a small number of (usually richer) countries,
although the exact pattern changes over time as new nations emerge and as the Games themselves evolve. Other work looks at age profiles
and participation by sex, showing that some sports are dominated by young athletes while others allow longer careers, and that the share of
female competitors has risen steadily over the twentieth century.

This notebook does not try to replicate those studies in full. Instead, the focus is on using the same underlying data to demonstrate sound
data‑management practice: making the dataset easier to handle, producing sensible summary statistics and then turning those into visual
summaries that would be understandable to a non‑technical audience.


In [ ]:
# 3. Data loading

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set some basic plotting defaults
plt.rcParams["figure.figsize"] = (8, 5)
sns.set_theme()

# Load the Olympic athlete–event data
olympics = pd.read_csv("athlete_events.csv")

print("Shape (rows, columns):", olympics.shape)
print("Column names:", olympics.columns.tolist())

olympics.head()


The file `athlete_events.csv` is loaded into a DataFrame called `olympics`. Each row represents a single athlete–event appearance, so the
same athlete can appear in multiple rows if they competed in several events or in more than one Games. The dataset contains 271,116 rows
and 15 columns covering identifiers, demographics, country codes (NOCs), events and medal outcomes.


In [ ]:
# 4. Data cleaning

# 4.1 Overview of missing values
missing_counts = olympics.isna().sum().sort_values(ascending=False)
missing_counts


In [ ]:
# 4.2 Remove exact duplicate rows (if any)
before_rows = len(olympics)
olympics = olympics.drop_duplicates()
after_rows = len(olympics)
removed_duplicates = before_rows - after_rows

print(f"Rows before removing duplicates: {before_rows}")
print(f"Rows after removing duplicates:  {after_rows}")
print(f"Exact duplicate rows removed:    {removed_duplicates}")


In [ ]:
# 4.3 Convert Year to a datetime representation for convenience
olympics["Year_dt"] = pd.to_datetime(olympics["Year"], format="%Y")

olympics.info()


## 3. Data management: cleaning

The basic cleaning steps are deliberately kept simple and transparent:

- The `isna()` summary shows that missing values are concentrated in the `Medal`, `Height`, `Weight` and `Age` columns. Missing medals are
  expected because most athletes do not win a medal. Missing age and anthropometric data are more problematic and are handled later in a
  way that depends on the analysis being performed.
- Exact duplicate rows across all columns are removed using `drop_duplicates()`. This avoids counting the same athlete–event twice. In the
  original file there are 1,385 such rows, which are dropped.
- A new column `Year_dt` is created by converting the integer `Year` to a datetime object. This does not change the underlying values but
  makes it easier to work with time‑based groupings if needed.

At this stage the dataset is structurally tidy enough to move on to feature construction.


In [ ]:
# 4.4 Simple imputation for height and weight (stretch element)

# Work on copies so that the original columns are preserved
olympics["Height_imputed"] = olympics["Height"]
olympics["Weight_imputed"] = olympics["Weight"]

# Compute median height and weight within Sport-Sex groups
group_height_median = olympics.groupby(["Sport", "Sex"])["Height"].transform("median")
group_weight_median = olympics.groupby(["Sport", "Sex"])["Weight"].transform("median")

# Fill missing values using these medians
olympics["Height_imputed"] = olympics["Height_imputed"].fillna(group_height_median)
olympics["Weight_imputed"] = olympics["Weight_imputed"].fillna(group_weight_median)

olympics[["Sport", "Sex", "Height", "Height_imputed", "Weight", "Weight_imputed"]].head()


For some later summaries it is convenient to have complete height and weight information. Rather than dropping all rows with missing
values, a simple form of imputation is used as a stretch element: for each sport–sex combination the median height and weight are
calculated and used to fill in any gaps. The original `Height` and `Weight` columns are retained so that it remains clear which values are
observed and which are imputed.


In [ ]:
# 5. Data wrangling

# 5.1 Age groups
age_bins = [0, 18, 25, 35, 200]
age_labels = ["0–18", "19–25", "26–35", "36+"]
olympics["Age_Group"] = pd.cut(olympics["Age"], bins=age_bins, labels=age_labels, include_lowest=True)
olympics["Age_Group"] = olympics["Age_Group"].cat.add_categories(["Unknown"])
olympics.loc[olympics["Age"].isna(), "Age_Group"] = "Unknown"

# 5.2 Tidy name fields
olympics["Full_Name"] = olympics["Name"].str.strip()
olympics["Given_Name"] = olympics["Full_Name"].str.split().str[0]
olympics["Family_Name"] = olympics["Full_Name"].str.split().str[-1]

# 5.3 Century and decade
olympics["Century_Number"] = (olympics["Year"] - 1) // 100 + 1

def to_ordinal(n: int) -> str:
    # Simple ordinal helper; suitable for small integers such as centuries
    return "%d%s" % (n, "tsnrhtdd"[(n // 10 % 10 != 1) * (n % 10 < 4) * n % 10 :: 4])

olympics["Century_Label"] = olympics["Century_Number"].apply(to_ordinal)
olympics["Decade"] = (olympics["Year"] // 10) * 10

# 5.4 Medal indicators
olympics["Is_Medalist"] = olympics["Medal"].notna().astype(int)
medal_points = {"Gold": 3, "Silver": 2, "Bronze": 1}
olympics["Medal_Score"] = olympics["Medal"].map(medal_points).fillna(0).astype(int)

olympics[["Age", "Age_Group", "Full_Name", "Century_Label", "Decade", "Is_Medalist", "Medal_Score"]].head()


## 4. Data management: wrangling

Several additional variables are created to make the data easier to analyse:

- **Age_Group**: athletes are placed into broad age bands (0–18, 19–25, 26–35, 36+). Observations without an age are labelled as
  `Unknown`. This allows age patterns to be compared across sports and over time without relying on exact ages.
- **Full_Name**, **Given_Name** and **Family_Name**: the original `Name` field is cleaned so that names can be used consistently when
  aggregating medals by athlete.
- **Century_Label** and **Decade**: these variables summarise the timing of each Games in a more interpretable way than the raw year and
  make it straightforward to look at long‑run trends.
- **Is_Medalist** and **Medal_Score**: these indicators distinguish medallists from non‑medallists and assign a simple score to medal
  types (Gold = 3, Silver = 2, Bronze = 1, no medal = 0). This is useful when summarising overall success across different dimensions.

These steps satisfy the core wrangling requirements specified in the brief and also introduce a couple of extra variables that are helpful
for later analysis.


In [ ]:
# 6. Data analysis

# 6.1 Average age of athletes for each Olympic event
age_by_event = (
    olympics
    .dropna(subset=["Age"])
    .groupby("Event", as_index=False)["Age"]
    .mean()
    .rename(columns={"Age": "Average_Age"})
)

# Identify youngest and oldest events
youngest_events = age_by_event.sort_values("Average_Age").head(10)
oldest_events = age_by_event.sort_values("Average_Age").tail(10)

print("Youngest events by average age:")
display(youngest_events)

print("Oldest events by average age:")
display(oldest_events)


In [ ]:
# 6.2 Top 10 countries with the most gold medals

gold_only = olympics[olympics["Medal"] == "Gold"]

gold_by_noc = (
    gold_only
    .groupby("NOC", as_index=False)
    .size()
    .rename(columns={"size": "Gold_Count"})
    .sort_values("Gold_Count", ascending=False)
)

top10_gold_noc = gold_by_noc.head(10)
top10_gold_noc


In [ ]:
# 6.3 Athlete with the most medals in each sport

medal_rows = olympics[olympics["Medal"].notna()]

medals_per_athlete_sport = (
    medal_rows
    .groupby(["Sport", "Full_Name"], as_index=False)
    .size()
    .rename(columns={"size": "Medal_Count"})
)

idx_max = medals_per_athlete_sport.groupby("Sport")["Medal_Count"].idxmax()
top_athletes_by_sport = medals_per_athlete_sport.loc[idx_max].sort_values("Medal_Count", ascending=False)

top_athletes_by_sport.head(15)


## 5. Data analysis

Three main analyses follow the assignment brief.

First, average age is calculated for each event, using only rows where `Age` is available. The youngest events tend to be in women's
swimming and rhythmic gymnastics, where the mean age is in the late teens. At the other end of the scale, some early archery team
competitions and the now‑discontinued art competitions have mean ages close to fifty. This confirms the intuitive idea that different
sports place peak physical demands at very different stages of an athlete's life.

Second, gold‑medal counts are aggregated by NOC and ranked. The United States sits comfortably at the top of the table, followed by the
Soviet Union and Germany. Great Britain, Italy, France, Sweden, Canada, Hungary and East Germany also appear in the top ten. The presence
of NOCs that no longer exist reminds us that simple medal tables blend together sporting performance and geopolitical history.

Third, within each sport the total number of medals is counted for each athlete. Taking the top performer in each sport highlights familiar
names such as Michael Phelps in swimming, Larysa Latynina in gymnastics and others who have dominated their disciplines over several
Games. This reinforces the idea that Olympic success is highly concentrated, not only at the country level but also at the individual level.


In [ ]:
# 7. Data visualisation

# 7.1 Top 10 NOCs by gold medals
plt.figure()
sns.barplot(data=top10_gold_noc, x="NOC", y="Gold_Count")
plt.title("Top 10 NOCs by number of gold medals")
plt.xlabel("NOC")
plt.ylabel("Number of gold medals")
plt.tight_layout()
plt.savefig("figure_top10_gold_noc.png", dpi=300)
plt.show()


In [ ]:
# 7.2 Age distribution in selected sports (boxplot)

selected_sports = ["Gymnastics", "Swimming", "Equestrianism", "Shooting"]
age_sport_sample = olympics.dropna(subset=["Age"])
age_sport_sample = age_sport_sample[age_sport_sample["Sport"].isin(selected_sports)]

plt.figure()
sns.boxplot(data=age_sport_sample, x="Sport", y="Age")
plt.title("Distribution of athlete age in selected sports")
plt.xlabel("Sport")
plt.ylabel("Age")
plt.tight_layout()
plt.savefig("figure_age_distribution_selected_sports.png", dpi=300)
plt.show()


In [ ]:
# 7.3 Gender participation over time (line chart)

entries_by_sex = olympics.pivot_table(
    index="Year",
    columns="Sex",
    values="ID",
    aggfunc="count"
)

plt.figure()
for sex_code, label in [("M", "Male"), ("F", "Female")]:
    if sex_code in entries_by_sex.columns:
        plt.plot(entries_by_sex.index, entries_by_sex[sex_code], label=label)

plt.legend()
plt.title("Olympic participation by sex over time")
plt.xlabel("Year")
plt.ylabel("Number of athlete–event entries")
plt.tight_layout()
plt.savefig("figure_gender_participation_over_time.png", dpi=300)
plt.show()


## 6. Visualisation and interpretation

The three figures produced from the analysis stage are intended to be simple but informative.

- **Top 10 NOCs by gold medals**: the bar chart underlines how skewed the distribution of gold medals is. The United States stands well
  clear of other countries, and there is a steep drop‑off after the first few NOCs. This is consistent with the idea that long‑run Olympic
  success is concentrated among a relatively small group of nations with the resources and sporting infrastructure to support repeated
  high‑level performance.
- **Age distributions in selected sports**: the boxplots for gymnastics and swimming show younger age distributions, with low medians and
  relatively narrow interquartile ranges. Equestrianism and shooting, by contrast, have higher median ages and much wider spreads,
  implying that athletes can remain competitive for longer. The visual makes clear at a glance how the physical and technical demands of
  different sports translate into very different career patterns.
- **Gender participation over time**: the line chart confirms that early Games were overwhelmingly male. Female participation rises slowly at
  first and then more rapidly from the latter half of the twentieth century onwards. Although male entries still slightly exceed female
  entries in the most recent Games in the dataset, the gap has narrowed considerably. This pattern reflects the gradual removal of formal
  barriers to women's participation and changing social attitudes towards women's sport.

Together, these visualisations demonstrate not only basic proficiency with Matplotlib and Seaborn but also the ability to pick plots that
support a clear narrative about the evolution of the Games.


In [ ]:
# 8. Export of results

# Save the cleaned and wrangled DataFrame
olympics.to_csv("athlete_events_cleaned.csv", index=False)

print("Cleaned dataset exported as 'athlete_events_cleaned.csv'.")
print("Figures saved as PNG files in the current working directory.")


## 7. Concluding remarks

This notebook has taken the original Olympic athlete–event dataset and worked through the data‑management and visualisation tasks set
out in the MN5813 group assignment. After loading the data, basic cleaning steps were used to remove duplicate rows and to prepare the
`Year` variable for time‑based analysis. A small number of additional variables were then created to capture age bands, temporal groupings
and simple measures of medal success.

The analysis itself showed that:

- average athlete age differs markedly between sports, with some disciplines dominated by teenagers and others accommodating much older
  competitors;
- historical gold medals are highly concentrated among a handful of NOCs, with the United States at the top, but with important
  contributions from now‑defunct states; and
- participation by sex has shifted from almost exclusively male in the early Games to a far more balanced picture in recent decades.

The accompanying visualisations turned these numerical summaries into more intuitive stories that could be shared with a non‑technical
audience. In combination, the cleaning, wrangling, analysis and plotting steps demonstrate how Pandas and standard Python libraries can be
used to make sense of a large but structured real‑world dataset.


## 8. Limitations and possible extensions

A few limitations of the exercise are worth noting. First, the dataset records medal outcomes but not the underlying performance margin,
so it is not possible to distinguish narrow defeats from decisive wins. Secondly, the Olympic programme has expanded and changed over
time, which affects medal opportunities and makes direct comparisons across eras more complicated than the simple summaries suggest.
Thirdly, the way NOCs map onto countries is not straightforward in all cases; states have split and merged, and some NOCs no longer exist.

Finally, although simple imputation was used for height and weight, more sophisticated approaches could be explored, and the impact of
missing data on particular analyses could be investigated in more detail.

There are several natural extensions. Medal totals could be normalised by population or income to give a sense of performance relative to
resources. The analysis could also be narrowed to focus on a single sport or region, or external data (for example, host‑nation effects or
macro‑economic indicators) could be added to model the determinants of Olympic success more explicitly.


## 9. References

- Kaggle, *120 years of Olympic history: athletes and results* (dataset used in MN5813 materials).  
- Bowden, J. (2011). *Writing a Report: How to Prepare, Write and Present Really Effective Reports*. How To Books.  

Additional academic references on Olympic performance, gender participation and sport analytics could be incorporated to expand the
literature background if required.
